In [19]:
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt
from gurobipy import GRB
import re
import datetime
import ast
from tqdm.notebook import tqdm

In [2]:
from tests.utils import *

## Master problem correctness

In [15]:
file_name = "test7_slack9"
n = 200
K = 5
seed = 5
l = 3
alpha = 1/2
beta = [1, 1, 1]
lower = 2
upper = n
M = 1e6
np.random.seed(seed)
labels = np.random.choice([0, 1, 2], p=[0.1, 0.2, 0.7], size=n)

X, Y, archetype = synthetic_data(n, K, seed)
_, _, q, clusters_init, _, t = initialize_clusters(
    X, K, l, alpha, beta, n, labels, M, lower, upper)

Optimizing Cluster Centers: 100%|██████████| 300/300 [00:00<00:00, 3239.96it/s, Status=SUCCESS]


In [4]:
for i in range(K):
    t_vector = []
    for j in range(l):
        cluster = np.array(clusters_init)[i,:n]
        size = sum(cluster)
        group_size = cluster @ q[j]
        if group_size >= alpha * size:
            t_vector.append(1)
        else:
            t_vector.append(0)
    print(t_vector)

NameError: name 'clusters_init' is not defined

In [21]:
np.array(t).T

array([[1, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [1, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1]])

In [5]:
model = gp.read("./tests/model_write/" + file_name + "_out1000.lp")
constrs_len = len(model.getConstrs())
print(constrs_len)
clusters = np.loadtxt("./tests/model_matrix/"+file_name+"_clusters.txt")
distances = np.loadtxt("./tests/model_matrix/"+file_name+"_distances.txt")
slacks = np.loadtxt("./tests/model_matrix/"+file_name+"_slacks.txt")
solutions = np.loadtxt("./tests/model_matrix/"+file_name+"_solutions.txt")
objectives = np.loadtxt("./tests/model_matrix/"+file_name+"_objectives.txt")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-08
Read LP format model from file ./tests/model_write/test7_slack9_out1000.lp
Reading time = 0.64 seconds
: 204 rows, 10184 columns, 815156 nonzeros
204


In [6]:
clusters = np.concatenate((clusters,np.ones(clusters.shape[0]).reshape(-1,1)),axis=1)
A = [[model.getCoeff(constr, var) for var in model.getVars()] for constr in model.getConstrs()]
b = [constr.rhs for constr in model.getConstrs()]
c = [var.obj for var in model.getVars()]
induced_mat = np.concatenate((clusters[:K,:],np.eye(constrs_len-1,constrs_len),clusters[K:,:]),axis=0).T
A = np.array(A)

In [7]:
np.allclose(induced_mat[:,:A.shape[1]],A)

True

In [8]:
dist_gurobi = np.array(c[:K] + c[K + constrs_len-1:])
dist_stored = np.array(distances)[:dist_gurobi.shape[0]]

In [9]:
np.allclose(dist_gurobi,dist_stored)

True

In [156]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 204 rows, 10184 columns and 815156 nonzeros
Model fingerprint: 0x363dd51f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 0 rows and 666 columns
Presolve time: 0.47s
Presolved: 204 rows, 9518 columns, 774170 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.061e+04
 Factor NZ  : 2.091e+04 (roughly 4 MB of memory)
 Factor Ops : 2.851e+06 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Comp

In [157]:
objectives[-1]

707.4135697279462

In [158]:
model.ObjVal

707.4135696685239

In [159]:
last_sol_gurobi = [var.X for var in model.getVars()]
last_sol_gurobi_Z = last_sol_gurobi[:K] + last_sol_gurobi[K + constrs_len-1:]
last_sol_gurobi_slackness = last_sol_gurobi[K:K + constrs_len-1]

check partition constraints

In [10]:
partition_constr = [clusters[:solutions[-1].shape[0],i] @ solutions[-1] + slacks[-1][i] for i in range(n)]

In [11]:
np.allclose(np.array(partition_constr),np.ones(n))

True

check number of clusters constraint

In [12]:
np.sum(solutions[-1]) == K

True

check fairness constraint

In [18]:
m = solutions[-1].shape[0]

In [172]:
distances[0]

886.9812193257434

In [178]:
clusters[0][:n]

array([1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [21]:
t1 = []
t2 = []
for i in (range(m)):
    t_vector = []
    for j in range(l):
        cluster = clusters[i,:n]
        size = sum(cluster)
        group_size = cluster @ q[j]
        if group_size >= alpha * size:
            t_vector.append(1)
        else:
            t_vector.append(0)
    t1.append(t_vector)
    t2.append(clusters[i,n:n+l])

In [28]:
np.allclose(t1[5:],t2[5:])

True

Gurobi doesn't generate the same solution for the 2 solves, and it may be caused by similar costs of clusters

In [102]:
np.argmax(solutions[-1] - last_sol_gurobi_Z)

9483

In [104]:
np.argmin(solutions[-1] - last_sol_gurobi_Z)

9494

In [106]:
distances[9494]

86.48909234264944

In [105]:
distances[9483]

86.48909234264944

In [98]:
np.max(slacks[-1] - last_sol_gurobi_slackness)

1.0591600102158837e-06

In [77]:
solutions[-1][9494]

0.0

In [78]:
last_sol_gurobi_Z[9494]

0.30371333887479834

## Visualizations

## Parameters

In [23]:
Zvars = model.getVars()[:K] + model.getVars()[K + constrs_len-1:]
Slack_vars_partition = model.getVars()[K : K + n]
Slack_vars_represent = model.getVars()[K + n : K + n + l]

In [24]:
len(Zvars) + len(Slack_vars_partition) + len(Slack_vars_represent) == len(model.getVars())

True

In [25]:
objective = gp.quicksum(Zvars[i] * dist_gurobi[i] for i in range(len(Zvars))) + \
    gp.quicksum(i * 0 for i in Slack_vars_partition) + \
    gp.quicksum(i * 0 for i in Slack_vars_represent)
model.setObjective(objective, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 204 rows, 10184 columns and 815156 nonzeros
Model fingerprint: 0x2d71cc41
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 4e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]


Presolve removed 3 rows and 869 columns
Presolve time: 0.42s
Presolved: 201 rows, 9315 columns, 765410 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 2.010e+04
 Factor NZ  : 2.030e+04 (roughly 4 MB of memory)
 Factor Ops : 2.727e+06 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.67 seconds (0.34 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       6    2.9098429e+01   0.000000e+00   0.000000e+00      1s

Solved in 6 iterations and 0.77 seconds (0.36 work units)
Optimal objective  2.909842886e+01


In [26]:
c2 = [var.obj for var in model.getVars()]

In [27]:
c2 == c

False

In [47]:
def get_nonzeros(l):
    res = []
    for i,j in enumerate(l):
        if j != 0:
            res.append(i)
    #print(l)
    print(res)